In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from utils import iter_df_read

# Modificar por ruta local de almacenamiento de datos
BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

# Operacionalización de datos

### Alumno desertor
- Tiene que haber pasado un tiempo mayor a la duración de la carrera (`cursando==0`)
- Considerar todo para el mismo codigo de carrera cada vez
- No debe estar en la base de titulados para esa carrera

### Tipos de deserción

- Sale del sistema y no vuelve a ingresar
- Sale del sistema y vuelve después de 1, 2 ó 3 años:
    - Vuelve a la misma carrera en la misma institución (no deserta, "congela")
    - Vuelve a otra carrera en la misma institución
    - Vuelve a la misma carrera en otra institución
    - Vuelve a la otra carrera en otra institución 

# Matriculados

## Lectura de datos matriculados

Fuente: https://datosabiertos.mineduc.cl/matricula-en-educacion-superior/

- Se considera todos los matriculados en CFT / IP desde 2015

In [2]:
# Matriculados de CFT / IP
mat_cft_ip = pd.read_csv(f"{BASE}/tmp_data/fechas_titulacion.csv")
mat_cft_ip.head()

,codigo_unico,mrun,fecha_obtencion_titulo
0,I111S2C413J1V1,15015557.0,20220228.0
1,I117S1C60J4V1,6100912.0,20220228.0
2,I260S50C108J1V1,2713742.0,20220228.0
3,I260S38C47J2V1,14478456.0,20220228.0
4,I260S3C280J1V1,23547583.0,20220228.0


In [3]:
mat_cft_ip.isna().sum()

codigo_unico                   0
mrun                           0
fecha_obtencion_titulo    815446
dtype: int64

In [4]:
# Se carga datos de matrícula para estos alumnos, considerando todas las instituciones
cols = [
    'cat_periodo', 'codigo_unico', 'mrun', 'anio_ing_carr_ori', 'anio_ing_carr_act',
    'tipo_inst_1', 'dur_estudio_carr', 'cod_inst', 'cod_carrera'
]

mruns = mat_cft_ip["mrun"].unique()

matriculados = pd.concat([
    iter_df_read(f, "mrun", mruns, sep=";", usecols=cols)
    for f in tqdm(
        glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"),
        total=8
    )
]).drop_duplicates().reset_index(drop=True)

  0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
# Se agrega columna que indica titulación

if "fecha_obtencion_titulo" not in matriculados.columns:
    len_or = matriculados.shape[0]
    matriculados = pd.merge(
        matriculados,
        mat_cft_ip,
        how="left"
    )
    
    assert matriculados.shape[0] == len_or

## Pre procesamiento: Año de titulación

In [7]:
matriculados["anio_titulacion"] = matriculados["fecha_obtencion_titulo"].fillna("").astype(str).str[:4].replace([""], [np.nan]).astype(float)
matriculados["anio_titulacion"].describe().round()

count    1416585.0
mean        2019.0
std            2.0
min         2015.0
25%         2017.0
50%         2019.0
75%         2020.0
max         2022.0
Name: anio_titulacion, dtype: float64

## Pre procesamiento: Cursando carrera

Se crea columna `cursando`, la cual indica:
- `1` si los años transcurridos desde el año de ingreso a la carrera hasta el año 2022 es menor o igual a la duración de la carrera en años.
- `0` en caso contrario (debió ya terminar la carrera)

Esto se utilizará más adelante en la construcción del vector objetivo; Se hace una selección de datos donde solo se selecciona a aquellos que tengan `cursando==0`, para medir inicialmente deserción solamente en aquellos que no se encuentren ya en la ventana de duración de la carrera.

In [8]:
matriculados["cursando"] = np.where(
    pd.Series([2022] * matriculados.shape[0]) - matriculados["anio_ing_carr_ori"]
    <= matriculados["dur_estudio_carr"] / 2, 1, 0
)

matriculados["cursando"].value_counts("%")

0    0.854918
1    0.145082
Name: cursando, dtype: float64

## Pre prcesamiento: Deserciones

### Primer caso: Sale del sistema y no vuelve a ingresar
Aquellos que cumplen a la vez que:
- Ha pasado el tiempo de su carrera
- Solo han tomado 1 carrera
- No se han titulado
- No se han matriculado el año 2022
- No tienen "años en blanco" (que salgan y vuelvan)

In [11]:
cft_ip_inst = ["Institutos Profesionales", "Centros de Formación Técnica"]

In [28]:
ruts_matriculados = matriculados[
    (matriculados["cursando"]==0)
    & (matriculados["tipo_inst_1"].isin(cft_ip_inst))
]["mrun"].unique()

print("Cant. matriculados CFT/IP que debieron terminar carrera (universo total): ", ruts_matriculados.shape[0])

Cant. matriculados CFT/IP que debieron terminar carrera (universo total):  1113239


In [17]:
ruts_no_titulados = matriculados[
    (matriculados["cursando"]==0)
    & (matriculados["tipo_inst_1"].isin(cft_ip_inst))
    & (matriculados["anio_titulacion"].isna())
]["mrun"].unique()

print("Cant. matriculados de CFT/IP que debieron terminar carrera y no se han titulado: ", ruts_no_titulados.shape[0])

Cant. matriculados de CFT/IP que debieron terminar carrear y no se han titulado:  675628


In [30]:
cant_carreras_no_tit = matriculados[
    matriculados["mrun"].isin(ruts_no_titulados)
][["mrun", "codigo_unico"]].drop_duplicates().groupby(["mrun"]).size().reset_index(name="cant_carreras")

ruts_no_tit_1_carr = cant_carreras_no_tit[cant_carreras_no_tit["cant_carreras"] == 1]["mrun"].unique()

print("Cant. matriculados de CFT/IP que debieron terminar carrera, no se han titulado, y solo han tomado 1 carrera: ", ruts_no_tit_1_carr.shape[0])

Cant. matriculados de CFT/IP que debieron terminar carrera, no se han titulado, y solo han tomado 1 carrera:  327910


In [40]:
ult_mat_no_tit_1_carr = matriculados[
    matriculados["mrun"].isin(ruts_no_tit_1_carr)
].groupby(["mrun"]).agg({
    "cat_periodo": "max"
}).reset_index()

ruts_no_tit_1_carr_no_mat_2022 = ult_mat_no_tit_1_carr[ult_mat_no_tit_1_carr["cat_periodo"] < 2022]["mrun"].unique()

print("Cant. matriculados de CFT/IP que debieron terminar carrera, no se han titulado, solo han tomado 1 carrera, y no estudian actualmente: ", ruts_no_tit_1_carr_no_mat_2022.shape[0])

Cant. matriculados de CFT/IP que debieron terminar carrera, no se han titulado, solo han tomado 1 carrera, y no estudian actualmente:  309391


In [65]:
max_dif_entre_mat = matriculados[
    matriculados["mrun"].isin(ruts_no_tit_1_carr_no_mat_2022)
].groupby(["mrun"]).agg({
    "cat_periodo": lambda x: np.diff(x).max() if len(np.diff(x)) else 0
}).reset_index()

ruts_desertores_1 = max_dif_entre_mat[max_dif_entre_mat["cat_periodo"] == 1]["mrun"].unique()

print(
    f"Cant. matriculados de CFT/IP que debieron terminar carrera, no se han titulado, solo han tomado 1 carrera, no estudian actualmente, y se han matriculado siempre años seguidos: "
    f"{ruts_desertores_1.shape[0]}"
)

Cant. matriculados de CFT/IP que debieron terminar carrera, no se han titulado, solo han tomado 1 carrera, no estudian actualmente, y se han matriculado siempre años seguidos: 130822


In [68]:
print(f"% Desertores Criterio 1: {np.round(ruts_desertores_1.shape[0] / ruts_matriculados.shape[0] * 100, 1)}%")

% Desertores Criterio 1: 11.8%


### Segundo caso:  Sale del sistema y vuelve a la misma carrera en la misma institución